In [1]:
import pandas as pd

pd.set_option('display.max_rows', 10)

# Create an empty DataFrame with the specified columns
df = pd.read_csv("data.csv", index_col=0)

# Show the empty DataFrame
data = df


In [2]:
# Replace NaN values with 0
data_filled = data.fillna(0)

# Ensure all columns except 'replicates' (since it might contain non-numeric values originally) are numeric
data_filled = data_filled.apply(pd.to_numeric, errors='ignore')

# Display the data types to confirm changes
data_filled.dtypes, data_filled.head()

data_filled


,round,replicates,duplicates,distortion,m1,m2,m3
0,-1,_21,100.0,1,1.000000,0.794769,0.885650
1,-1,_21,8.0,1,1.000000,0.688249,0.815341
2,-1,_21,1.0,0,0.997807,0.919192,0.956888
3,-1,_21,0.1,0,0.967742,0.952381,0.960000
4,-1,_21,100.0,0,1.000000,0.921762,0.959288
...,...,...,...,...,...,...,...
2905,-12,_50,100.0,1,0.997545,0.817572,0.898636
2906,-12,_50,0.1,1,1.000000,0.783333,0.878505
2907,-12,_50,1.0,1,0.987113,0.786448,0.875429
2908,-12,_50,0.1,0,0.967742,0.952381,0.960000


In [3]:
# Remove "_" from the 'replicates' column and convert to numeric if possible
data_filled['replicates'] = data_filled['replicates'].astype(str).str.replace("_", "")
data_filled['replicates'] = pd.to_numeric(data_filled['replicates'], errors='coerce').fillna(0)

# Make 'round' and 'split' values positive
data_filled['round'] = data_filled['round'].abs()


# Rename columns 'm1', 'm2', 'm3' to 'precision', 'recall', 'f1score'
data_renamed = data_filled.rename(columns={'m1': 'precision', 'm2': 'recall', 'm3': 'f1score'})

# Display the first few rows to confirm the column names have been updated
data_renamed


,round,replicates,duplicates,distortion,precision,recall,f1score
0,1,21,100.0,1,1.000000,0.794769,0.885650
1,1,21,8.0,1,1.000000,0.688249,0.815341
2,1,21,1.0,0,0.997807,0.919192,0.956888
3,1,21,0.1,0,0.967742,0.952381,0.960000
4,1,21,100.0,0,1.000000,0.921762,0.959288
...,...,...,...,...,...,...,...
2905,12,50,100.0,1,0.997545,0.817572,0.898636
2906,12,50,0.1,1,1.000000,0.783333,0.878505
2907,12,50,1.0,1,0.987113,0.786448,0.875429
2908,12,50,0.1,0,0.967742,0.952381,0.960000


In [8]:
# Define a function to map 'round' values to model names
def map_round_to_model(round_number):
    if 1 <= round_number <= 3:
        return "Both"
    elif 4 <= round_number <= 6:
        return "No Diri"
    elif 7 <= round_number <= 9:
        return "None"
    elif 10 <= round_number <= 12:
        return "No Empirical"
    else:
        return "Unknown"  # For any round numbers outside the specified ranges

# Apply the mapping function to the 'round' column to create the new 'model' column
data_renamed['model'] = data_renamed['round'].apply(map_round_to_model)



# Define a function to map 'round' values to prior categories
def map_round_to_prior(round_number):
    if round_number in [1, 4, 7, 10]:
        return "Pitman"
    elif round_number in [2, 5, 8, 11]:
        return "uniform"
    elif round_number in [3, 6, 9, 12]:
        return "Bounded NBD"
    else:
        return "Unknown"  # For any round numbers outside the specified or considered ranges

# Apply the mapping function to the 'round' column to create the new 'prior' column
data_renamed['prior'] = data_renamed['round'].apply(map_round_to_prior)

# Define a function to map 'duplicates' values to duplicates level categories
def map_duplicates_to_level(duplicates_value):
    if duplicates_value == 0.1:
        return "Low"
    elif duplicates_value == 1.0:
        return "Medium"
    elif duplicates_value == 8.0:
        return "High"
    elif duplicates_value == 100.0:
        return "Very High"
    else:
        return "Unknown"  # For any duplicates values outside the specified ranges

# Apply the mapping function to the 'duplicates' column to create the new 'duplicates_level' column
data_renamed['duplicates_level'] = data_renamed['duplicates'].apply(map_duplicates_to_level)

# Define a function to map 'distortion' values to distortion level categories
def map_distortion_to_level(distortion_value):
    if distortion_value == 0:
        return "Low"
    elif distortion_value == 1:
        return "High"
    else:
        return "Unknown"  # For any distortion values outside the specified values

# Apply the mapping function to the 'distortion' column to create the new 'distortion_level' column
data_renamed['distortion_level'] = data_renamed['distortion'].apply(map_distortion_to_level)

def map_iter_to_num(iter):
    if iter == '10k':
        return 10000
    elif iter == '50k':
        return 50000
    else:
        return 0  # For any distortion values outside the specified values


# Select only the specified columns
data_selected_columns = data_renamed[['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates']]

# Select and reorder the dataset with only the specified columns
data_final = data_renamed[['model', 'prior', 'duplicates_level', 'distortion_level',
                           'replicates', 'precision', 'recall', 'f1score']]

# Display the first few rows to confirm the changes
data_final






,model,prior,duplicates_level,distortion_level,replicates,precision,recall,f1score
0,Both,Pitman,Very High,High,21,1.000000,0.794769,0.885650
1,Both,Pitman,High,High,21,1.000000,0.688249,0.815341
2,Both,Pitman,Medium,Low,21,0.997807,0.919192,0.956888
3,Both,Pitman,Low,Low,21,0.967742,0.952381,0.960000
4,Both,Pitman,Very High,Low,21,1.000000,0.921762,0.959288
5,Both,Pitman,High,Low,21,1.000000,0.936901,0.967423
6,Both,Pitman,Medium,High,21,0.988950,0.735113,0.843345
7,Both,Pitman,Low,High,21,1.000000,0.750000,0.857143
8,Both,Pitman,High,High,22,1.000000,0.685052,0.813093
9,Both,Pitman,Very High,High,22,0.997204,0.717639,0.834633


In [5]:
data_cleaned = data_final

# Finding duplicates and keeping them to show
duplicates_to_show = data_cleaned[data_cleaned.duplicated(subset=['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates'])]

# Sorting the duplicates for better readability
duplicates_sorted = duplicates_to_show.sort_values(by=['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates'])

pd.set_option('display.max_rows', None)

duplicates_sorted


,model,prior,duplicates_level,distortion_level,replicates,precision,recall,f1score
518,Both,Bounded NBD,High,High,22,0.996862,0.761791,0.863616
532,Both,Bounded NBD,High,High,23,1.000000,0.764988,0.866848
524,Both,Bounded NBD,Low,High,22,1.000000,0.750000,0.857143
529,Both,Bounded NBD,Low,High,23,1.000000,0.750000,0.857143
523,Both,Bounded NBD,Medium,High,22,0.991735,0.739220,0.847059
534,Both,Bounded NBD,Medium,High,23,0.989011,0.739220,0.846063
520,Both,Bounded NBD,Very High,High,22,0.997654,0.855801,0.921300
533,Both,Bounded NBD,Very High,High,23,0.997565,0.824279,0.902681
13,Both,Pitman,High,High,22,1.000000,0.768985,0.869408
38,Both,Pitman,High,High,24,1.000000,0.766587,0.867873


In [6]:
data_cleaned = data_cleaned.drop_duplicates(subset=['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates'], keep='last')
data_cleaned.reset_index(drop=True, inplace=True)
data_cleaned.shape

(2874, 8)

In [16]:
# Select the columns of interest
columns_of_interest = ['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates']
df_selected = data_cleaned[columns_of_interest]


# Option 2: Count across selected columns (if you want a total count regardless of the column)
total_count = df_selected[['model', 'prior', 'replicates']].value_counts() # Sorting by index (the number) for better readability
print("Total count across selected columns:")
print(total_count)

Total count across selected columns:
model         prior        replicates
Both          Bounded NBD  21            8
No Empirical  Pitman       36            8
              uniform      26            8
                           25            8
                           24            8
                           23            8
                           22            8
                           21            8
              Pitman       50            8
                           49            8
                           48            8
                           47            8
                           46            8
                           45            8
                           44            8
                           43            8
                           42            8
                           41            8
                           40            8
                           39            8
                           38            8
              uniform 

In [7]:
# Checking for missing combinations in the first 6 columns
# First, we create a DataFrame of all possible combinations of these columns

# Get unique values for each of the first 6 columns
model_values = data_cleaned['model'].unique()
prior_values = data_cleaned['prior'].unique()
duplicates_level_values = data_cleaned['duplicates_level'].unique()
distortion_level_values = data_cleaned['distortion_level'].unique()
num_iter_values = data_cleaned['num_iter'].unique()
replicates_values = data_cleaned['replicates'].unique()

# Use itertools.product to create all possible combinations
from itertools import product
all_combinations = list(product(model_values, prior_values, duplicates_level_values, distortion_level_values, num_iter_values, replicates_values))

# Convert to DataFrame
all_combinations_df = pd.DataFrame(all_combinations, columns=['model', 'prior', 'duplicates_level', 'distortion_level', 'num_iter', 'replicates'])

# Now, find missing combinations by checking which rows in all_combinations_df are not present in data_cleaned_no_duplicates
missing_combinations = pd.merge(all_combinations_df, data_cleaned, on=['model', 'prior', 'duplicates_level', 'distortion_level', 'num_iter', 'replicates'], how='left', indicator=True).query('_merge == "left_only"').drop(columns=['precision', 'recall', 'f1score', '_merge'])

missing_combinations_count = missing_combinations.shape[0]

print(missing_combinations_count)
pd.set_option('display.max_rows', None)
missing_combinations


KeyError: 'num_iter'

In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
# Group by the specified columns and compute mean and standard deviation for precision, recall, and f1score
grouped_data = data_final.groupby(['model', 'prior', 'duplicates_level', 'distortion_level', 'num_iter'])

# Compute mean
mean_values = grouped_data[['precision', 'recall', 'f1score']].mean().reset_index()

# Compute standard deviation
std_dev_values = grouped_data[['precision', 'recall', 'f1score']].std().reset_index()

# Rename columns for clarity
mean_values.rename(columns={
    'precision': 'precision_mean', 
    'recall': 'recall_mean', 
    'f1score': 'f1score_mean'}, inplace=True)

std_dev_values.rename(columns={
    'precision': 'precision_std', 
    'recall': 'recall_std', 
    'f1score': 'f1score_std'}, inplace=True)

# Display the mean values
mean_values.head(), std_dev_values.head()


In [ ]:
mean_values.shape

In [ ]:
# Merge mean and standard deviation dataframes on the groupby columns
combined_data = pd.merge(mean_values, std_dev_values, 
                         on=['model', 'prior', 'duplicates_level', 'distortion_level', 'num_iter'])


pd.set_option('display.max_rows', None)


# Display the combined dataframe
combined_data


In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
combined_data.to_csv("clean_data.csv")